In [23]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import os
import json

In [2]:
base_url = 'https://carigold.com/forum/'
r = requests.get(base_url)

In [11]:
skip = ['politics.108', 'santai-others.106', 'official-news-current-issues.72']

In [12]:
soup = BeautifulSoup(r.content, "lxml")
a = soup.find_all('a', {'class': 'subNodeLink'})
a = [a_.get('href') for a_ in a]
a = [a_ for a_ in a if not any([s in a_ for s in skip])]

In [16]:
!mkdir topics

In [27]:
max_worker = 20

def get_href(url):
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    try:
        max_page = max([int(a_.split('page-')[-1]) for a_ in a if '/forum/forums/' in a_ and '/page-' in a_])
    except:
        max_page = -1
    a = [a_ for a_ in a if a_.startswith('/forum/threads/')]
    filtered = []
    for a_ in a:
        splitted = a_.split('/')
        if len(splitted) == 5 and splitted[-1] == '':
            filtered.append(a_)
    return filtered, max_page

In [30]:
for i in tqdm(range(len(a))):
    a_ = a[i]
    filename = os.path.join('topics', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    base_url = f'https://carigold.com{a_}'
    
    filtered, max_page = get_href(base_url)
    
    for i in tqdm(range(2, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{base_url}page-{a}' for a in aranged]

        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(get_href, url): url for url in urls}

            for future in as_completed(futures):
                filtered.extend(future.result()[0])
                
    topics = list(set(filtered))
    with open(filename, 'w') as fopen:
        json.dump(topics, fopen)

 72%|██████████████████████████████▏           | 95/132 [02:50<01:05,  1.77s/it]


 19%|████████▏                                 | 23/118 [04:09<16:36, 10.49s/it]
0it [00:00, ?it/s]
 23%|█████████▌                                | 27/118 [04:26<11:17,  7.45s/it]
0it [00:00, ?it/s]
 25%|██████████▋                               | 30/118 [04:32<06:49,  4.65s/it]
0it [00:00, ?it/s]
 27%|███████████▍                              | 32/118 [04:35<04:48,  3.36s/it]
0it [00:00, ?it/s]
 89%|████████████████████████████████████████     | 8/9 [00:13<00:01,  1.75s/it]


 35%|██████████████▌                           | 41/118 [05:21<07:34,  5.91s/it]
0it [00:00, ?it/s]
 41%|█████████████████                         | 48/118 [06:15<09:06,  7.81s/it]
0it [00:00, ?it/s]
 45%|██████████████████▊                       | 53/118 [07:17<19:51, 18.34s/it]
0it [00:00, ?it/s]
 20%|████████▊                                   | 2/10 [00:04<00:16,  2.12s/it]


 78%|█████████████████████████████████▍         | 28/36 [00:49<00:14,  1.79s/it]


 49%|█████████████████████                      | 25/51 [00:48<00:46,  1.78s/it]


...

 62%|██████████████████████████▋                | 44/71 [01:18<00:46,  1.73s/it]


100%|███████████████████████████████████████████| 68/68 [02:03<00:00,  1.82s/it]


 33%|██████████████▋                             | 8/24 [00:14<00:28,  1.80s/it]


 11%|████▋                                       | 3/28 [00:05<00:44,  1.76s/it]


100%|█████████████████████████████████████████| 118/118 [30:30<00:00, 15.51s/it]
